In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn

In [ ]:
data = pd.read_excel('C:/Buffer/06-all-basis.xlsx')

In [ ]:
data

In [ ]:
uniq_names = data['StrategyName'].unique()
uniq_strats = {}
for i in uniq_names:
  uniq_strats[i] = data[data['StrategyName'] == i]
  uniq_strats[i]['TradeTime'] = uniq_strats[i]['TradeTime'].apply(pd.to_datetime)
  uniq_strats[i].sort_values('TradeTime', ascending = False)
  #uniq_strats[i]['Direction'].replace(to_replace = 'Sell', value = -1, inplace = True)
  #uniq_strats[i]['Direction'].replace(to_replace = 'Buy', value = 1, inplace = True)
  #uniq_strats[i]['Direction'] = uniq_strats[i]['Direction'].apply(int)
  uniq_strats[i] = uniq_strats[i][(uniq_strats[i]['TradeTime']>=pd.to_datetime('2021-06-30'))&(uniq_strats[i]['TradeTime']<pd.to_datetime('2021-07-01'))]
  #print(uniq_strats[i]['TradeTime'])
  #print( uniq_strats[i]['Direction'])
  #uniq_strats[i]['basis'] += np.max(uniq_strats[i]['basis'])+1

In [ ]:
Profits = {}
feerate = 0.001
for i in uniq_names:
  if len(uniq_strats[i])==0:
    continue
  capital = [0]
  avprices = [0]
  profits = [0]
  profit_times = [(uniq_strats[i].iloc[-1]['TradeTime'])]
  volumes = [0]
  therest = 0
  counter = 0
  for t in range(len(uniq_strats[i])-1, -1, -1): 

      if volumes[-1]>=0 and uniq_strats[i].iloc[t]['Direction'] > 0:
        volumes.append(volumes[-1]+uniq_strats[i].iloc[t]['Volume'])
        avprices.append(avprices[-1]*volumes[-2]/volumes[-1]+uniq_strats[i].iloc[t]['Volume']*uniq_strats[i].iloc[t]['basis']/volumes[-1])
        continue
      if volumes[-1]>0 and uniq_strats[i].iloc[t]['Direction'] < 0:
        profit_times.append(uniq_strats[i].iloc[t]['TradeTime'])
        volumes.append(volumes[-1]-uniq_strats[i].iloc[t]['Volume'])
        if volumes[-1]<=0:
          profits.append((uniq_strats[i].iloc[t]['basis']-avprices[-1])*volumes[-2])
          avprices.append(-uniq_strats[i].iloc[t]['basis'])
        else:
          profits.append((uniq_strats[i].iloc[t]['basis']-avprices[-1])*uniq_strats[i].iloc[t]['Volume']) 
          avprices.append(avprices[-1])
        continue

      if volumes[-1]<=0 and uniq_strats[i].iloc[t]['Direction'] < 0:
        volumes.append(volumes[-1]-uniq_strats[i].iloc[t]['Volume'])
        avprices.append(avprices[-1]*volumes[-2]/volumes[-1]+uniq_strats[i].iloc[t]['Volume']*uniq_strats[i].iloc[t]['basis']/volumes[-1])
        continue
      if volumes[-1]<0 and uniq_strats[i].iloc[t]['Direction'] > 0:
        profit_times.append(uniq_strats[i].iloc[t]['TradeTime'])
        volumes.append(volumes[-1]+uniq_strats[i].iloc[t]['Volume'])
        if volumes[-1]>=0:
          profits.append((uniq_strats[i].iloc[t]['basis']+avprices[-1])*volumes[-2])
          avprices.append(uniq_strats[i].iloc[t]['basis'])
        else:
          profits.append((-uniq_strats[i].iloc[t]['basis']-avprices[-1])*uniq_strats[i].iloc[t]['Volume'])
          avprices.append(avprices[-1])
        continue
  uniq_strats[i]['avprices']=np.flip(avprices[1:])  
  uniq_strats[i]['vols']=np.flip(volumes[1:])  
  Profits[i] = pd.DataFrame({'times':pd.to_datetime(profit_times), 'profits':profits})

  print(i)
  print(Profits[i])

In [ ]:
for i in uniq_names:
  print(i,sum(Profits[i]['profits']))

In [ ]:
Profits['ETH_USDT_SWAP_04%'][Profits['ETH_USDT_SWAP_04%']['times']>=pd.to_datetime('2021-06-30')]

In [ ]:
for i in uniq_names:
  fo j

In [ ]:
#np.mean(uniq_strats['ETH_USDT_SWAP_04%'][uniq_strats['ETH_USDT_SWAP_04%']['Direction']==-1]['basis'])
plt.figure(figsize=(5,5))
uniq_strats['ETH_USDT Bi-OK_0.1%']['basis'].iloc[np.argmax(uniq_strats['ETH_USDT Bi-OK_0.1%']['basis'])] = 0
print(uniq_strats['ETH_USDT_SWAP_01%'][uniq_strats['ETH_USDT_SWAP_01%']['Direction']=='Buy']['basis'])
seaborn.histplot(uniq_strats['ETH_USDT Bi-OK_0.1%'][uniq_strats['ETH_USDT Bi-OK_0.1%']['Direction']=='Buy']['basis'].iloc[:-1], bins = 50, color = 'blue')
seaborn.histplot(uniq_strats['ETH_USDT Bi-OK_0.1%'][uniq_strats['ETH_USDT Bi-OK_0.1%']['Direction']=='Sell']['basis'].iloc[:-1], bins = 50, color = 'green')

In [ ]:
sum(uniq_strats['ETH sell'][-21:]['basis']*uniq_strats['ETH sell'][-21:]['Volume'])/sum(uniq_strats['ETH sell'][-21:]['Volume'])

In [ ]:
win = {}
means = {}
loose = {}
names = []
for i in uniq_names:
  if len(uniq_strats[i])>0:
    names.append(i)
    win[i] = {'buy':[0,0],'sell':[0,0]}
    k = uniq_strats[i][uniq_strats[i]['Direction'] == 'Buy']
    if len(k)>0:
      win[i]['buy'][0] = len(k[k['basis']>0])
      win[i]['buy'][1] = len(k[k['basis']<=0])

    k = uniq_strats[i][uniq_strats[i]['Direction'] == 'Sell']
    if len(k)>0:
      win[i]['sell'][0] = len(k[k['basis']>0])
      win[i]['sell'][1] = len(k[k['basis']<=0])

In [ ]:
for i in names:
    print(i,'\t', win[i]['buy'][0],'\t',win[i]['buy'][1],'\t',win[i]['sell'][0],'\t',win[i]['sell'][1] )